In [5]:
import pandas as pd
import plotly.graph_objects as go

# Sample data creation (replace with actual jobs data)
class JobsData:
    def __init__(self, data):
        data = {
            'Date': pd.date_range(start='2023-01-01', end='2024-12-01', freq='M'),
            'Jobs': [150000 + x * 5000 for x in range(24)]
        }
    def dataCompile(self):
        df = pd.DataFrame(data)
        df['Monthly Change'] = df['Jobs'].diff()
        df['Quarterly Change'] = df['Jobs'].diff(3)
        df['Annual Change'] = df['Jobs'].diff(12)

# Create plots for monthly, quarterly, and annual changes

    def plot(fig = go.Figure()):
        fig.add_trace(go.Scatter(
        x=df['Date'], y=df['Monthly Change'], mode='lines+markers', name='Monthly Change'))

        fig.add_trace(go.Scatter(
        x=df['Date'], y=df['Quarterly Change'], mode='lines+markers', name='Quarterly Change'))

        fig.add_trace(go.Scatter(
        x=df['Date'], y=df['Annual Change'], mode='lines+markers', name='Annual Change'))

# Add titles and layout
fig.update_layout(
    title="Jobs Numbers Over Time",
    xaxis_title="Date",
    yaxis_title="Jobs Change",
    legend_title="Change Type"
)

# Save the plot to an HTML file
fig.write_html("jobs_dashboard.html")

# Instructions:
# 1. Commit the `jobs_dashboard.html` file to your GitHub repository.
# 2. Enable GitHub Pages in your repository settings, pointing to the main branch.
# 3. Access your dashboard at `https://your-username.github.io/your-repository/`.
